In [36]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import cv2
from fastai.vision import *
import glob, os
import pandas as pd
from tqdm import tqdm

model_dir = "models"
model_name = "efficientnet-b0-224-fp32.pkl"
path = Path("../../task1/photos")

learn = load_learner(model_dir,model_name)

In [52]:
img = open_image("../../task1/photos/holdout/hex/8798_2.jpg")
a = learn.predict(img)
(cat, _, (pr1, pr2)) = a
print(a)
print(str(cat), float(pr1), float(pr2))

(Category hex, tensor(0), tensor([9.9981e-01, 1.9320e-04]))
hex 0.999806821346283 0.00019320323190186173


In [2]:
img_size=224
seed = 42
tfms = get_transforms(flip_vert=False,p_affine=0,max_lighting=0.4,max_rotate=5)

In [3]:
data = ImageDataBunch.from_folder(path,  test='holdout',
                                  ds_tfms=tfms, size=img_size, num_workers=16,
                                  seed=seed).normalize(imagenet_stats)

/opt/tljh/user/lib/python3.7/site-packages/fastai/data_block.py:457: UserWarning: Your validation set is empty. If this is by design, use `split_none()`
                 or pass `ignore_empty=True` when labelling to remove this warning.
  or pass `ignore_empty=True` when labelling to remove this warning.""")


In [6]:
df = pd.DataFrame(columns=['target','img','pred','hex','tet'])
num = len(data.test_ds)
for i in tqdm(range(num)):
    target = filename = str(data.test_ds.items[i]).split('/')[-2]
    filename = str(data.test_ds.items[i]).split('/')[-1]
    predict = learn.predict(data.test_ds[i][0])
    df.loc[i] = [ target, filename, str(predict[0]), to_np(predict[2][0]), to_np(predict[2][1]) ]
df

100%|██████████| 656/656 [02:55<00:00,  3.73it/s]


,target,img,pred,hex,tet
0,tet,7897_3_{X-14}.jpg,tet,3.053436e-05,0.9999695
1,tet,7618_4_{II-17}.jpg,tet,2.5149113e-06,0.9999975
2,tet,8364_1_{X-14}.jpg,tet,0.0046575856,0.9953425
3,tet,8663_4_{X-14}.jpg,tet,0.00021522914,0.99978477
4,tet,7894_3_{X-14}.jpg,tet,0.00012312212,0.99987686
...,...,...,...,...,...
651,hex,7792_2_{IX-18}.jpg,hex,0.9996113,0.00038869889
652,hex,6527_4.jpg,hex,1.0,5.840186e-08
653,hex,6937_2.jpg,hex,0.99999845,1.5860833e-06
654,hex,8798_2.jpg,hex,0.9998068,0.00019320323


In [9]:
from sklearn.metrics import accuracy_score

In [10]:
accuracy_score(df.target,df.pred)

0.9878048780487805

In [27]:
df['y_true'] = df.target.apply(lambda x: 0 if x == 'hex' else 1)
df['y_pred'] = df.apply(lambda x: x.tet if x.pred == 'tet' else 1-x.hex, axis=1)

In [28]:
df

,target,img,pred,hex,tet,y_true,y_pred
0,tet,7897_3_{X-14}.jpg,tet,3.053436e-05,0.9999695,1,0.999969
1,tet,7618_4_{II-17}.jpg,tet,2.5149113e-06,0.9999975,1,0.999997
2,tet,8364_1_{X-14}.jpg,tet,0.0046575856,0.9953425,1,0.995342
3,tet,8663_4_{X-14}.jpg,tet,0.00021522914,0.99978477,1,0.999785
4,tet,7894_3_{X-14}.jpg,tet,0.00012312212,0.99987686,1,0.999877
...,...,...,...,...,...,...,...
651,hex,7792_2_{IX-18}.jpg,hex,0.9996113,0.00038869889,0,0.000389
652,hex,6527_4.jpg,hex,1.0,5.840186e-08,0,0.000000
653,hex,6937_2.jpg,hex,0.99999845,1.5860833e-06,0,0.000002
654,hex,8798_2.jpg,hex,0.9998068,0.00019320323,0,0.000193


In [29]:
from sklearn.metrics import roc_auc_score
roc_auc_score(df.y_true, df.y_pred)

0.9953472866419294

In [30]:
df[df['target'] != df['pred'] ]

,target,img,pred,hex,tet,y_true,y_pred
24,tet,8817_3_{II17}.jpg,hex,0.9680988,0.03190123,1,0.031901
25,tet,8817_{II17}.jpg,hex,0.99995136,4.865126e-05,1,0.000049
46,tet,8817_1_{II17}.jpg,hex,0.9999957,4.249099e-06,1,0.000004
93,tet,8817_2_{II17}.jpg,hex,0.99992526,7.472128e-05,1,0.000075
163,tet,8817_4_{II17}.jpg,hex,0.9999758,2.4190367e-05,1,0.000024
350,hex,7798_{IX-18}.jpg,tet,0.4260234,0.57397664,0,0.573977
444,hex,6766_{IX-18}.jpg,tet,0.116613716,0.8833863,0,0.883386
557,hex,6734_{IX-18}.jpg,tet,0.3182252,0.6817748,0,0.681775
